# Setup
import all the libraries and functions we will need ..

In [1]:
import json                             # to store data in JSON format

from numpy import array, argmin         # to get min distance between known face and unknown face

import os, glob                         # to take all images from directory

import cv2                              # for image processing
import face_recognition                 # for face detection, encoding and recognition

from playsound import playsound         # to play alarm sound when face is not recognized


# Store known Data
some lists to store information about the people we want to know from the video, wheteher they are them or not ..

In [2]:
known_face_encodings    = []            # to store known face encodings
frame_resize            = 0.25          # to resize frame

### load multi users

In [3]:
all_images = glob.glob(os.path.join("images 1/*.*"))                # get all images from directory

for image in all_images:                                            # for each image
    image = cv2.imread(image)                                       # read image

    small_img = cv2.resize(image, (0, 0), fx = frame_resize, fy = frame_resize)   # resize image  (0.25 original size)
    image_encoding = face_recognition.face_encodings(small_img)[0]  # get encoding


    known_face_encodings.append(image_encoding)

# Detect faces function
get all faces and their names from in the frame and compare them using distance 

the minimum distance between him and the known people **will most likely be correct**


In [ ]:
def detect_faces_from_frame(frame, is_detected) -> bool:
    
    small_frame = cv2.resize(frame, (0, 0), fx = frame_resize, fy = frame_resize)

    face_locations = face_recognition.face_locations(small_frame)                       # get face locations

    face_encodings = face_recognition.face_encodings(small_frame, face_locations)       # get face encodings

    for face_encoding in face_encodings:                                                # for each face in the frame if there are multiple faces in the frame

        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)   # compare face encodings with known face encodings, return True if they match

        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)    # get min distance between known face and unknown face

        best_match_index = argmin(face_distances)                                       # get index of min distance

        if matches[best_match_index]:                                                   # if unknown face matches with known face
            is_detected = True                                   # write True

    face_locations = array(face_locations)
                                                  # convert to array to do operations quickly (using list will not work)
    return is_detected

# Test 

In [14]:
cap = cv2.VideoCapture(0)                        # camera setup   

same_persone = False                             # initialize shared variable

frame_num = 0                                    # to count frames

while frame_num < 60:                            # 30 frames is enough to detect face

    ret, frame = cap.read()                      # read frame by frame

    same_persone = detect_faces_from_frame(frame, same_persone)               # using detect faces function to get face locations and names in the frame

    cv2.imshow("Frame", frame)                   # show frame after processing

    # same_persone = read_shared_variable()        # read shared variable

    print(f'frame number: {frame_num + 1} - verified: {same_persone}')
    frame_num += 1

    if same_persone == True:                     # if it's True (unknown face matches with known face) 
        break

    if cv2.waitKey(1) & 0xFF == ord('q'):        # press q to quit
        break

cap.release()                                    # release camera
cv2.destroyAllWindows()                          # close all windows (imshow windows)

if same_persone == False:                        # if it's False (unknown face does not matches with known face)
    print("playing alarm")
    playsound('alarm.wav')


frame number: 1 - verified: True
